In [3]:
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
import seaborn as sns
import ast

#display settings
pd.set_option('display.float_format','{:.6f}'.format)

In [7]:
file_path="TRADES_CopyTr_90D_ROI.csv"
df = pd.read_csv(file_path)
print(df.head())


              Port_IDs                                      Trade_History
0  3925368433214965504  [{'time': 1718899656000, 'symbol': 'SOLUSDT', ...
1  4002413037164645377  [{'time': 1718980078000, 'symbol': 'NEARUSDT',...
2  3923766029921022977  [{'time': 1718677164000, 'symbol': 'ETHUSDT', ...
3  3994879592543698688  [{'time': 1718678214000, 'symbol': 'ETHUSDT', ...
4  3926423286576838657  [{'time': 1718979615000, 'symbol': 'ETHUSDT', ...


In [9]:
df.info() # shows column names,data types , and missing values
df.describe() #summary statistics of numerical columns 
df.columns #list all column names

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150 entries, 0 to 149
Data columns (total 2 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   Port_IDs       150 non-null    int64 
 1   Trade_History  149 non-null    object
dtypes: int64(1), object(1)
memory usage: 2.5+ KB


Index(['Port_IDs', 'Trade_History'], dtype='object')

In [11]:
# droping rows where value is missing
df.dropna(subset=['Trade_History'],inplace=True)

In [13]:
#converting trade_history from string to dic
df['Trade_History']=df['Trade_History'].apply(lambda x:ast.literal_eval(x) if isinstance(x,str) else x)

In [15]:
#separating dataframes from trade history

trade_data = []

for index, row in df.iterrows():
    port_id = row['Port_IDs']
    trades = row['Trade_History']
    
    for trade in trades:
        trade_data.append({
            'Port_IDs': port_id,
            'Time': trade.get('time', None),
            'Symbol': trade.get('symbol', None),
            'Side': trade.get('side', None),
            'Price': trade.get('price', None),
            'Quantity': trade.get('quantity', None),
            'Realized_Profit': trade.get('realizedProfit', None)
        })

# Converting the list into a DataFrame
trade_df = pd.DataFrame(trade_data)

# Show first few rows
trade_df.head()
                           

,Port_IDs,Time,Symbol,Side,Price,Quantity,Realized_Profit
0,3925368433214965504,1718899656000,SOLUSDT,BUY,132.537000,1988.055000,0.000000
1,3925368433214965504,1718899618000,DOGEUSDT,BUY,0.121820,1398.980880,0.000000
2,3925368433214965504,1718899618000,DOGEUSDT,BUY,0.121820,197.470220,0.000000
3,3925368433214965504,1718899616000,DOGEUSDT,BUY,0.121820,16.567520,0.000000
4,3925368433214965504,1718899616000,DOGEUSDT,BUY,0.121820,92.217740,0.000000


In [21]:
# return on investment
roi_df = trade_df.groupby('Port_IDs').agg(ROI=('Realized_Profit', lambda x: (x.sum() / trade_df.loc[x.index, 'Quantity'].sum()) * 100)).reset_index()

In [23]:
# calculating profit and loss
pnl_df = trade_df.groupby('Port_IDs').agg(PnL=('Realized_Profit', 'sum')).reset_index()

In [25]:
#calculating win rate and win positions
win_positions_df = trade_df[trade_df['Realized_Profit'] > 0].groupby('Port_IDs').agg(Win_Positions=('Realized_Profit', 'count')).reset_index()
total_positions_df = trade_df.groupby('Port_IDs').agg(Total_Positions=('Realized_Profit', 'count')).reset_index()
win_rate_df = win_positions_df.merge(total_positions_df, on='Port_IDs')
win_rate_df['Win_Rate'] = (win_rate_df['Win_Positions'] / win_rate_df['Total_Positions']) * 100

In [27]:
# calculating max drawdown
def max_drawdown(profits):
    peak = profits.cummax()
    drawdown = (profits - peak) / peak
    return drawdown.min() * 100  # Convert to percentage

mdd_df = trade_df.groupby('Port_IDs').agg(MDD=('Realized_Profit', max_drawdown)).reset_index()

In [29]:
# calculating sharpe ratio
def sharpe_ratio(profits):
    mean_return = profits.mean()
    std_dev = profits.std()
    return mean_return / std_dev if std_dev != 0 else 0  # Avoid division by zero

sharpe_df = trade_df.groupby('Port_IDs').agg(Sharpe_Ratio=('Realized_Profit', sharpe_ratio)).reset_index()

In [31]:
#merging all metrics into table
final_df = roi_df.merge(pnl_df, on='Port_IDs') \
                 .merge(win_rate_df[['Port_IDs', 'Win_Rate', 'Win_Positions', 'Total_Positions']], on='Port_IDs') \
                 .merge(mdd_df, on='Port_IDs') \
                 .merge(sharpe_df, on='Port_IDs')

# Sort by ROI and PnL for ranking
final_df = final_df.sort_values(by=['ROI', 'PnL'], ascending=False).reset_index(drop=True)

# Get Top 20 Accounts
top_20_accounts = final_df.head(20)

In [33]:
top_20_accounts.to_csv('Top_20_Accounts.csv', index=False)

In [35]:
top_20_accounts

,Port_IDs,ROI,PnL,Win_Rate,Win_Positions,Total_Positions,MDD,Sharpe_Ratio
0,3826087012661391104,12.178517,532.656974,58.333333,63,108,-133.409633,0.675101
1,3768170840939476993,8.779089,243.668899,42.857143,6,14,-100.000000,0.424277
2,3977234346014419201,4.102464,2427.292831,39.759036,33,83,-100.000000,0.304736
3,4029507714735307777,3.422201,1537.899960,3.773585,2,53,-100.000000,0.195852
4,4023697881318718465,3.307649,3151.950620,8.571429,6,70,-100.000000,0.275870
5,4004713168329653760,3.210949,1667.198610,7.407407,2,27,-100.000000,0.277411
6,4030708912786799105,3.201021,1612.843770,3.333333,2,60,-100.000000,0.184141
7,4023697433751327232,3.186034,2408.224000,9.523810,4,42,-100.000000,0.300159
8,4029749871687083265,3.170541,3662.136548,42.873832,367,856,-100.000000,0.353877
9,4039129759104249600,3.162479,1264.289200,44.360902,59,133,-100.000000,0.522043
